### Downloading dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def load_datasets(dataset_names, base_path):
    datasets = {}
    for name in dataset_names:
        file_path = os.path.join(base_path, name + '.csv')
        try:
            datasets[name] = pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            datasets[name] = None
    return datasets


In [ ]:
import os

# Assuming /kaggle/input/brazilian-ecommerce/ is the correct directory
base_path = '/kaggle/input/brazilian-ecommerce/'

# Dataset names
dataset_names = [
    'olist_customers_dataset', 
    'olist_order_items_dataset', 
    'olist_order_payments_dataset', 
    'olist_order_reviews_dataset', 
    'olist_orders_dataset', 
    'olist_sellers_dataset', 
    'olist_products_dataset', 
    'product_category_name_translation'
]

# Assuming you have a function load_datasets that loads the datasets given the names and path
datasets = load_datasets(dataset_names, base_path)

# Displaying the first few rows of each dataset to understand their structure
for name, df in datasets.items():
    if df is not None:
        print(f"\nFirst few rows of {name}:")
        print(df.head())
    else:
        print(f"Failed to load {name}")

In [ ]:
import pandas as pd

def merge_datasets(datasets):
    # Start by merging orders with customers
    merged = pd.merge(datasets['olist_orders_dataset'], datasets['olist_customers_dataset'], on='customer_id', how='left')

    # Add other datasets with the correct merge keys
    merge_keys = {
        'order_items': 'order_id',
        'order_payments': 'order_id',
        'order_reviews': 'order_id',
        'sellers': 'seller_id',
        'products': 'product_id',
        'product_category_name_translation': 'product_category_name'  # Make sure this dataset is loaded correctly
    }

    for name, key in merge_keys.items():
        if name in datasets:
            merged = pd.merge(merged, datasets[name], on=key, how='left')

    return merged

import pandas as pd

# Load individual datasets
olist_customers_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
order_items_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')
order_reviews_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
olist_orders_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
sellers_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
products_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
product_category_name_translation_df = pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')

# Define the datasets dictionary
datasets = {
    'olist_customers_dataset': olist_customers_df,
    'order_items': order_items_df,
    'order_payments': order_payments_df,
    'order_reviews': order_reviews_df,
    'olist_orders_dataset': olist_orders_df,
    'sellers': sellers_df,
    'products': products_df,
    'product_category_name_translation': product_category_name_translation_df
}

# Now, you can call the merge_datasets function with the loaded datasets
merged_df = merge_datasets(datasets)

### Data cleaning

In [ ]:
# applying data cleaning to new csv file
merged_df.to_csv('olist_merged_data.csv', index=False)
merged_df.info()

In [ ]:
# check for duplicates
merged_df.duplicated().sum()

In [ ]:
# check for missing values by percentage in each column
merged_df.isnull().sum() / len(merged_df) * 100

In [ ]:
# drop missing values column with more than 50% missing values
merged_df = merged_df.dropna(thresh=len(merged_df) * 0.5, axis=1)

# drop rows with missing values
merged_df = merged_df.dropna()

In [ ]:
# drop missing values column with more than 50% missing values
merged_df = merged_df.dropna(thresh=len(merged_df) * 0.5, axis=1)

# drop rows with missing values
merged_df = merged_df.dropna()

In [ ]:
# Clean and preprocess data
def preprocess_data(df):
    # Drop columns with more than 50% missing values
    df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)
    
    # Convert datetime columns
    datetime_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 
                    'order_delivered_customer_date', 'order_estimated_delivery_date', 
                    'shipping_limit_date', 'review_creation_date', 'review_answer_timestamp']
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Calculate new features
    df['time_to_delivery'] = (df['order_delivered_customer_date'] - df['order_approved_at']).dt.days
    df['order_processing_time'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.days
    df['estimated_vs_actual_shipping'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.days
    df['product_volume_m3'] = (df['product_length_cm'] * df['product_width_cm'] * df['product_height_cm']) / 1000000
    df['satisfaction'] = (df['review_score'] >= 4).astype(int)
    df['order_value'] = df['price'] + df['freight_value']

    # create late delivery flag
    df['late_delivery'] = (df['order_delivered_customer_date'] > df['order_estimated_delivery_date']).astype(int)


    # Drop rows with missing values
    df.dropna(inplace=True)

    # create seasonal features from order_purchase_timestamp
    df['order_month'] = df['order_purchase_timestamp'].dt.month
    df['order_day'] = df['order_purchase_timestamp'].dt.dayofweek
    df['order_hour'] = df['order_purchase_timestamp'].dt.hour

    return df

merged_df = preprocess_data(merged_df)

In [ ]:
# drop unnecessary columns
merged_df.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'review_score', 'seller_zip_code_prefix']
               , axis=1, inplace=True) 
# save the cleaned dataset
merged_df.to_csv('olist_merged_data_clean.csv', index=False)

In [ ]:
# check summary statistics
merged_df.describe()